In [1]:
import tensorflow as tf
import simpleaudio as sa
from scipy.signal import resample

In [2]:
file_path = 'data/audio/SMC_013.wav'

audio = tf.io.read_file(file_path)
audio, sample_rate = tf.audio.decode_wav(audio)

In [3]:
def resample_signal(x, src_rate, tgt_rate):
    new_total_samples = int(tgt_rate * len(x.numpy()) / src_rate.numpy())
    return resample(x, new_total_samples)

In [4]:
resampled_audio = resample_signal(audio, sample_rate, 11025)

In [12]:
sa.play_buffer(resampled_audio, 1, 4, 11025)
sa.play_buffer(audio, 1, 4, 44100)

In [6]:
Lx = 110 #Lx = 11025
Ly = 44 #Ly = 4410

baseline = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(256, dropout=0.1, recurrent_dropout=0.1, input_shape=(Lx, 1)),
    tf.keras.layers.Dense(Ly, activation='sigmoid')
])

baseline.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dense_1 (Dense)              (None, 44)                11308     
Total params: 275,500
Trainable params: 275,500
Non-trainable params: 0
_________________________________________________________________


In [7]:
X = tf.random.normal((1024, Lx, 1))
y = tf.random.uniform((1024, Ly), 0, 2, dtype=tf.int32)

In [9]:
baseline.compile(optimizer='rmsprop', loss='binary_crossentropy')
baseline.fit(X, y, epochs=2, batch_size=32)

Train on 1024 samples
Epoch 1/2
1024/1024 [==============================] - 14s 14ms/sample - loss: 0.6927
Epoch 2/2
1024/1024 [==============================] - 14s 14ms/sample - loss: 0.6924


In [13]:
preds = baseline.predict(X[0:1, :, :])

In [15]:
beats = tf.where(tf.math.greater(preds, 0.5), tf.constant(1), tf.constant(0))

In [16]:
beats

<tf.Tensor: shape=(1, 44), dtype=int32, numpy=
array([[1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0]],
      dtype=int32)>

In [ ]:
# x = read_audio
# y = read_tags
def create_label_tensor(x, tags):
    pass #return tensor with the same as x with 0/1s of the beats

create_label_tensor(x, y)